In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np

In [0]:
flight = pd.read_csv('gdrive/My Drive/Colab Notebooks/raw-flight-data.csv')
len(flight)

2719418

Hapus Data NULL

In [0]:
flight = flight[np.isfinite(flight['ArrDelay'])]
flight= flight[np.isfinite(flight['DepDelay'])]
print(flight.isnull().sum())
print(len(flight))

DayofMonth         0
DayOfWeek          0
Carrier            0
OriginAirportID    0
DestAirportID      0
DepDelay           0
ArrDelay           0
dtype: int64
2690385


Hapus Data Duplicates

In [0]:
duplicateRowsDF = flight[flight.duplicated()]
print("Duplicate Rows except first occurrence based on all columns are :")
print(len(duplicateRowsDF))
flight = flight.drop_duplicates(keep='first')
duplicateRowsDF = flight[flight.duplicated()]
print("Duplicate Rows except first occurrence based on all columns are :(After Drop)")
print(len(duplicateRowsDF))

Duplicate Rows except first occurrence based on all columns are :
17200
Duplicate Rows except first occurrence based on all columns are :(After Drop)
0


In [0]:
len(flight)

2673185

**Remove Outlier**

In [0]:
def outlier_treatment(datacolumn):
    sorted(datacolumn)
    Q1,Q3 = np.percentile(datacolumn , [25,75])
    IQR = Q3 - Q1
    # print(Q1)
    # print(Q3)    
    lower_range = Q1 - (1.5 * IQR)
    upper_range = Q3 + (1.5 * IQR)
    return lower_range,upper_range,IQR

In [0]:
l,u,iqr = outlier_treatment(flight['DepDelay'])
# print(l)
# print(u)
# print(iqr)


In [0]:
flight.drop(flight[ (flight['DepDelay'] > u) | (flight['DepDelay'] < l) ].index , inplace=True)
len(flight)

2360184

In [0]:
ll,uu,iqr2 = outlier_treatment(flight['ArrDelay'])
flight.drop(flight[ (flight['ArrDelay'] > uu) | (flight['ArrDelay'] < ll) ].index , inplace=True)
len(flight)

2284134

OneHotEncoding

In [0]:
cols = ['DayOfWeek','Carrier','OriginAirportID','DestAirportID']
newFlight = pd.get_dummies(flight[cols].astype(str))

In [0]:
newFlight = pd.concat([newFlight,flight[['DepDelay','ArrDelay']]], axis=1)
newFlight.head()

,DayOfWeek_1,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5,DayOfWeek_6,DayOfWeek_7,Carrier_9E,Carrier_AA,Carrier_AS,Carrier_B6,Carrier_DL,Carrier_EV,Carrier_F9,Carrier_FL,Carrier_HA,Carrier_MQ,Carrier_OO,Carrier_UA,Carrier_US,Carrier_VX,Carrier_WN,Carrier_YV,OriginAirportID_10140,OriginAirportID_10299,OriginAirportID_10397,OriginAirportID_10423,OriginAirportID_10529,OriginAirportID_10693,OriginAirportID_10721,OriginAirportID_10792,OriginAirportID_10800,OriginAirportID_10821,OriginAirportID_11042,OriginAirportID_11057,OriginAirportID_11066,OriginAirportID_11193,OriginAirportID_11259,OriginAirportID_11278,OriginAirportID_11292,...,DestAirportID_13198,DestAirportID_13204,DestAirportID_13232,DestAirportID_13244,DestAirportID_13303,DestAirportID_13342,DestAirportID_13487,DestAirportID_13495,DestAirportID_13796,DestAirportID_13830,DestAirportID_13851,DestAirportID_13871,DestAirportID_13891,DestAirportID_13930,DestAirportID_13931,DestAirportID_14027,DestAirportID_14057,DestAirportID_14100,DestAirportID_14107,DestAirportID_14122,DestAirportID_14307,DestAirportID_14492,DestAirportID_14524,DestAirportID_14570,DestAirportID_14635,DestAirportID_14679,DestAirportID_14683,DestAirportID_14730,DestAirportID_14747,DestAirportID_14771,DestAirportID_14831,DestAirportID_14843,DestAirportID_14869,DestAirportID_14893,DestAirportID_14908,DestAirportID_15016,DestAirportID_15304,DestAirportID_15376,DepDelay,ArrDelay
0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-3.0,1.0
1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,-8.0
2,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,-4.0,-15.0
3,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28.0,24.0
4,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-6.0,-11.0


Slice data

In [0]:
# slicing data
y = newFlight['ArrDelay']
x = newFlight.drop('ArrDelay', axis=1)

Import Library

In [0]:
from sklearn.model_selection import KFold
cv = KFold(n_splits=5)

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
# from sklearn.svm import SVR, LinearSVR
from sklearn.ensemble import RandomForestRegressor
# from sklearn.gaussian_process import GaussianProcessRegressor
# from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
# from sklearn.preprocessing import PolynomialFeatures

Kalau Jalanin ini artinya tnpa fitur seleksi

XGBoost

Fitur Seleksi dengan Pvalues => backward Elim

Fitur Selection kedua

In [0]:
# Linear Regression
score = []

for train_index, test_index in cv.split(x,y):
    model = LinearRegression()    
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]    
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    # y_train_predicted = model.predict(x_train)
     # evaluating the model on training dataset
    # rmse_train = np.sqrt(mean_squared_error(y_train, y_train_predicted))
    # r2_train = r2_score(y_train, y_train_predicted)
    
    # evaluating the model on test dataset
    # rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
    # r2_test = r2_score(y_test, y_pred)
    # print("The model performance for the training set")
    # print("-------------------------------------------")
    # print("RMSE of training set is {}".format(rmse_train))
    # print("R2 score of training set is {}".format(r2_train))
    
    # print("\n")
    
    # print("The model performance for the test set")
    # print("-------------------------------------------")
    # print("RMSE of test set is {}".format(rmse_test))
    # print("R2 score of test set is {}".format(r2_test))
    #metric performance calculate
    mse = mean_squared_error(y_test, y_pred)
    mse = mse**0.5
    score.append([mse])    
    print("MSE:", mse)
print(np.mean(score,axis=0))

MSE: 10.26727823449456
MSE: 9.704289554136079
MSE: 9.950503526904688
MSE: 9.545235654562658
MSE: 9.921865271781027
[9.87783445]


In [0]:
# XGbost Regression
import xgboost as xgb
from sklearn.metrics import mean_squared_error
score = []
params = {'objective' :'reg:linear','n_estimators': 10, 'max_depth': 5, 'min_samples_split': 2,
          'learning_rate': 0.1, 'loss': 'ls'}
# xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
#                 max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg = xgb.XGBRegressor(**params)
for train_index, test_index in cv.split(x,y): 
    #print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    xg_reg.fit(x_train, y_train)
    y_pred = xg_reg.predict(x_test)
    #metric performance
    mse = mean_squared_error(y_test, y_pred)
    mse = mse**0.5
    score.append([mse])    
    print("MSE:", mse)
print(np.mean(score,axis=0))

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[12:45:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MSE: 10.824143470120847


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[12:47:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MSE: 10.261356422153058


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[12:50:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MSE: 10.489126971844243


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[12:52:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MSE: 10.360214323394253


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[12:55:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MSE: 10.442664584875851
[10.47550115]


MSE: 10.6450
MSE: 10.1874
MSE: 10.4576
MSE: 10.1608
MSE: 10.2537
[10.3408982]


In [0]:
##random forest regression

score = []

for train_index, test_index in cv.split(x,y):
    model = RandomForestRegressor(n_estimators=10, max_depth=5, random_state=3)
    
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    
    mse = mean_squared_error(y_test, y_pred)
    mse = mse**0.5    
    
    score.append([mse])
    
    print("MSE:", mse)
print(np.mean(score, axis=0))

MSE: 10.40186945608757
MSE: 9.889863572069476
MSE: 10.184763549922017
MSE: 9.802224748117316
MSE: 9.981686604882004
[10.05208159]
